In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#load LoRA to train the model
%%capture
!pip install transformers peft bitsandbytes datasets accelerate

In [ ]:
##load huggingface dataset
%%capture
!pip install datasets
from datasets import load_dataset

# Load Deepseek model

Load directly:

1. More flexible, allows custom data processing

2. Suitable for fine-tuning and custom training

3. Supports multiple tasks (e.g., prompt engineering, reinforcement learning fine-tuning)

4. Allows manual adjustment of tokenizer and model parameters

Can use pipeline as a baseline and compare its performance with a model fine-tuned on our own data.

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

# Evaluate LLM before training

### Language Understanding& Knowledge Level: MMLU Evaluate

In [ ]:
%%capture
!pip install lm-eval
from lm_eval import evaluator
from lm_eval.models.huggingface import HFLM

In [ ]:
##load the mmlu dataset
ds = load_dataset("cais/mmlu", "all")

README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/3.50M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/408k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/76.5k [00:00<?, ?B/s]

auxiliary_train-00000-of-00001.parquet:   0%|          | 0.00/47.5M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/14042 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1531 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/285 [00:00<?, ? examples/s]

Generating auxiliary_train split:   0%|          | 0/99842 [00:00<?, ? examples/s]

In [ ]:
print(ds)  # print out dataset info
print(ds.keys())
print(ds["test"][0]) #print out the test info

DatasetDict({
    test: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 14042
    })
    validation: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 1531
    })
    dev: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 285
    })
    auxiliary_train: Dataset({
        features: ['question', 'subject', 'choices', 'answer'],
        num_rows: 99842
    })
})
dict_keys(['test', 'validation', 'dev', 'auxiliary_train'])
{'question': 'Find the degree for the given field extension Q(sqrt(2), sqrt(3), sqrt(18)) over Q.', 'subject': 'abstract_algebra', 'choices': ['0', '4', '2', '6'], 'answer': 1}


In [ ]:
#wrapped it in `lm_eval.models.huggingface.HFLM(pretrained=my_model)` first, then lm_eval can recognize the model
%%capture
lm_model = HFLM(model)

In [ ]:
tasks = ["mmlu_philosophy"]
# Evaluate
result_philosophy = evaluator.simple_evaluate(
    model=lm_model,
    tasks=tasks,
    batch_size=8)

print(result_philosophy)

Running loglikelihood requests: 100%|██████████| 1244/1244 [07:37<00:00,  2.72it/s]


{'results': {'mmlu_philosophy': {'alias': 'philosophy', 'acc,none': 0.41479099678456594, 'acc_stderr,none': 0.027982680459759556}}, 'group_subtasks': {'mmlu_philosophy': []}, 'configs': {'mmlu_philosophy': {'task': 'mmlu_philosophy', 'task_alias': 'philosophy', 'tag': 'mmlu_humanities_tasks', 'dataset_path': 'hails/mmlu_no_train', 'dataset_name': 'philosophy', 'dataset_kwargs': {'trust_remote_code': True}, 'test_split': 'test', 'fewshot_split': 'dev', 'doc_to_text': '{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:', 'doc_to_target': 'answer', 'unsafe_code': False, 'doc_to_choice': ['A', 'B', 'C', 'D'], 'description': 'The following are multiple choice questions (with answers) about philosophy.\n\n', 'target_delimiter': ' ', 'fewshot_delimiter': '\n\n', 'fewshot_config': {'sampler': 'first_n'}, 'num_fewshot': 0, 'metric_list': [{'metric': 'acc', 'aggregation': 'mean', 'higher_is_better': True}], 'output_type': 'multiple_choice', 

In [ ]:
tasks = ["mmlu_moral_scenarios"]

# Evaluate
result_moral_scenarios = evaluator.simple_evaluate(
    model=lm_model,
    tasks=tasks,
    batch_size=8)

print(result_moral_scenarios)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
tasks = ["mmlu_logical_fallacies"]

# Evaluate
result_logical_fallacies = evaluator.simple_evaluate(
    model=lm_model,
    tasks=tasks,
    batch_size=8)

print(result_logical_fallacies)

Running loglikelihood requests: 100%|██████████| 652/652 [03:52<00:00,  2.81it/s]


{'results': {'mmlu_logical_fallacies': {'alias': 'logical_fallacies', 'acc,none': 0.4110429447852761, 'acc_stderr,none': 0.038656978537853624}}, 'group_subtasks': {'mmlu_logical_fallacies': []}, 'configs': {'mmlu_logical_fallacies': {'task': 'mmlu_logical_fallacies', 'task_alias': 'logical_fallacies', 'tag': 'mmlu_humanities_tasks', 'dataset_path': 'hails/mmlu_no_train', 'dataset_name': 'logical_fallacies', 'dataset_kwargs': {'trust_remote_code': True}, 'test_split': 'test', 'fewshot_split': 'dev', 'doc_to_text': '{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:', 'doc_to_target': 'answer', 'unsafe_code': False, 'doc_to_choice': ['A', 'B', 'C', 'D'], 'description': 'The following are multiple choice questions (with answers) about logical fallacies.\n\n', 'target_delimiter': ' ', 'fewshot_delimiter': '\n\n', 'fewshot_config': {'sampler': 'first_n'}, 'num_fewshot': 0, 'metric_list': [{'metric': 'acc', 'aggregation': 'mean', 'higher

In [ ]:
tasks = ["mmlu_nutrition"]

# Evaluate
result_nutrition = evaluator.simple_evaluate(
    model=lm_model,
    tasks=tasks,
    batch_size=8)

print(result_nutrition)

Running loglikelihood requests: 100%|██████████| 1224/1224 [07:16<00:00,  2.81it/s]


{'results': {'mmlu_nutrition': {'alias': 'nutrition', 'acc,none': 0.4117647058823529, 'acc_stderr,none': 0.028180596328259293}}, 'group_subtasks': {'mmlu_nutrition': []}, 'configs': {'mmlu_nutrition': {'task': 'mmlu_nutrition', 'task_alias': 'nutrition', 'tag': 'mmlu_other_tasks', 'dataset_path': 'hails/mmlu_no_train', 'dataset_name': 'nutrition', 'dataset_kwargs': {'trust_remote_code': True}, 'test_split': 'test', 'fewshot_split': 'dev', 'doc_to_text': '{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:', 'doc_to_target': 'answer', 'unsafe_code': False, 'doc_to_choice': ['A', 'B', 'C', 'D'], 'description': 'The following are multiple choice questions (with answers) about nutrition.\n\n', 'target_delimiter': ' ', 'fewshot_delimiter': '\n\n', 'fewshot_config': {'sampler': 'first_n'}, 'num_fewshot': 0, 'metric_list': [{'metric': 'acc', 'aggregation': 'mean', 'higher_is_better': True}], 'output_type': 'multiple_choice', 'repeats': 1, 

In [ ]:
tasks = ["mmlu_sociology"]

# Evaluate
result_sociology = evaluator.simple_evaluate(
    model=lm_model,
    tasks=tasks,
    batch_size=8)

print(result_sociology)

Running loglikelihood requests: 100%|██████████| 804/804 [04:41<00:00,  2.86it/s]


{'results': {'mmlu_sociology': {'alias': 'sociology', 'acc,none': 0.4626865671641791, 'acc_stderr,none': 0.035256751674679745}}, 'group_subtasks': {'mmlu_sociology': []}, 'configs': {'mmlu_sociology': {'task': 'mmlu_sociology', 'task_alias': 'sociology', 'tag': 'mmlu_social_sciences_tasks', 'dataset_path': 'hails/mmlu_no_train', 'dataset_name': 'sociology', 'dataset_kwargs': {'trust_remote_code': True}, 'test_split': 'test', 'fewshot_split': 'dev', 'doc_to_text': '{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:', 'doc_to_target': 'answer', 'unsafe_code': False, 'doc_to_choice': ['A', 'B', 'C', 'D'], 'description': 'The following are multiple choice questions (with answers) about sociology.\n\n', 'target_delimiter': ' ', 'fewshot_delimiter': '\n\n', 'fewshot_config': {'sampler': 'first_n'}, 'num_fewshot': 0, 'metric_list': [{'metric': 'acc', 'aggregation': 'mean', 'higher_is_better': True}], 'output_type': 'multiple_choice', 'rep

In [ ]:
tasks = ["mmlu_medical_genetics"]

# Evaluate
result_medical_genetics = evaluator.simple_evaluate(
    model=lm_model,
    tasks=tasks,
    batch_size=8)

print(result_medical_genetics)

Running loglikelihood requests: 100%|██████████| 400/400 [02:05<00:00,  3.20it/s]


{'results': {'mmlu_medical_genetics': {'alias': 'medical_genetics', 'acc,none': 0.38, 'acc_stderr,none': 0.04878317312145633}}, 'group_subtasks': {'mmlu_medical_genetics': []}, 'configs': {'mmlu_medical_genetics': {'task': 'mmlu_medical_genetics', 'task_alias': 'medical_genetics', 'tag': 'mmlu_other_tasks', 'dataset_path': 'hails/mmlu_no_train', 'dataset_name': 'medical_genetics', 'dataset_kwargs': {'trust_remote_code': True}, 'test_split': 'test', 'fewshot_split': 'dev', 'doc_to_text': '{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:', 'doc_to_target': 'answer', 'unsafe_code': False, 'doc_to_choice': ['A', 'B', 'C', 'D'], 'description': 'The following are multiple choice questions (with answers) about medical genetics.\n\n', 'target_delimiter': ' ', 'fewshot_delimiter': '\n\n', 'fewshot_config': {'sampler': 'first_n'}, 'num_fewshot': 0, 'metric_list': [{'metric': 'acc', 'aggregation': 'mean', 'higher_is_better': True}], 'output

In [ ]:
tasks = ["mmlu_human_sexuality"]

# Evaluate
result_human_sexuality = evaluator.simple_evaluate(
    model=lm_model,
    tasks=tasks,
    batch_size=8)

print(result_human_sexuality)

Running loglikelihood requests: 100%|██████████| 524/524 [02:46<00:00,  3.14it/s]


{'results': {'mmlu_human_sexuality': {'alias': 'human_sexuality', 'acc,none': 0.4351145038167939, 'acc_stderr,none': 0.04348208051644858}}, 'group_subtasks': {'mmlu_human_sexuality': []}, 'configs': {'mmlu_human_sexuality': {'task': 'mmlu_human_sexuality', 'task_alias': 'human_sexuality', 'tag': 'mmlu_social_sciences_tasks', 'dataset_path': 'hails/mmlu_no_train', 'dataset_name': 'human_sexuality', 'dataset_kwargs': {'trust_remote_code': True}, 'test_split': 'test', 'fewshot_split': 'dev', 'doc_to_text': '{{question.strip()}}\nA. {{choices[0]}}\nB. {{choices[1]}}\nC. {{choices[2]}}\nD. {{choices[3]}}\nAnswer:', 'doc_to_target': 'answer', 'unsafe_code': False, 'doc_to_choice': ['A', 'B', 'C', 'D'], 'description': 'The following are multiple choice questions (with answers) about human sexuality.\n\n', 'target_delimiter': ' ', 'fewshot_delimiter': '\n\n', 'fewshot_config': {'sampler': 'first_n'}, 'num_fewshot': 0, 'metric_list': [{'metric': 'acc', 'aggregation': 'mean', 'higher_is_better':

### Long Context Understanding: ms_marco
Using ROUGE to evaluate the respone

In [ ]:
ds2 = load_dataset("microsoft/ms_marco", "v1.1")

README.md:   0%|          | 0.00/9.48k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/175M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

In [ ]:
print(ds2)  # print out dataset info
print(ds2.keys())
print(ds2["train"][0]) #print out the train info

DatasetDict({
    validation: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 10047
    })
    train: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 82326
    })
    test: Dataset({
        features: ['answers', 'passages', 'query', 'query_id', 'query_type', 'wellFormedAnswers'],
        num_rows: 9650
    })
})
dict_keys(['validation', 'train', 'test'])
{'answers': ['Results-Based Accountability is a disciplined way of thinking and taking action that communities can use to improve the lives of children, youth, families, adults and the community as a whole.'], 'passages': {'is_selected': [0, 0, 0, 0, 0, 1, 0, 0, 0, 0], 'passage_text': ["Since 2007, the RBA's outstanding reputation has been affected by the 'Securency' or NPA scandal. These RBA subsidiaries were involved in bribing overseas officials so that Australia might win luc

In [ ]:
##define train, test, validation set
train_data = ds2["train"]
test_data = ds2["test"]
validation_data = ds2["validation"]

In [ ]:
from tqdm import tqdm
import torch
from rouge_score import rouge_scorer

def evaluate_model_rouge2(model, tokenizer, test_data, max_samples=200):
    model.eval()
    device = model.device
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)

    rouge1_scores, rouge2_scores, rougeL_scores = [], [], []
    total = min(len(test_data), max_samples)

    for i in tqdm(range(total)):
        sample = test_data[i]
        question = sample["query"]

        if len(sample["answers"]) == 0:
            continue

        reference = sample["answers"][0]

        inputs = tokenizer(
            question,
            return_tensors="pt",
            max_length=512,
            truncation=True
        ).to(device)

        with torch.no_grad():
            outputs = model.generate(
                input_ids=inputs.input_ids,
                attention_mask=inputs.attention_mask,
                max_new_tokens=256,
                pad_token_id=tokenizer.eos_token_id
            )

        prediction = tokenizer.decode(outputs[0], skip_special_tokens=True)

        scores = scorer.score(reference, prediction)
        rouge1_scores.append(scores["rouge1"].fmeasure)
        rouge2_scores.append(scores["rouge2"].fmeasure)
        rougeL_scores.append(scores["rougeL"].fmeasure)

    return {
        "ROUGE-1": sum(rouge1_scores) / total,
        "ROUGE-2": sum(rouge2_scores) / total,
        "ROUGE-L": sum(rougeL_scores) / total
    }

In [ ]:
## run evalution
results = evaluate_model_rouge2(model, tokenizer, test_data)
print(results)

# Inference with baseline model

In [ ]:
# Play with the model: without any training, see what can it generate
model.config.pad_token_id = model.config.eos_token_id

# let model generate text
def generate_response(prompt, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt")  # transform to tensor
    input_ids = inputs["input_ids"]  # get input_ids
    attention_mask = inputs["attention_mask"]  # get attention_mask
    with torch.no_grad():  # close the gradient calculate, save memory
        output = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=max_length)  # generate text
    return tokenizer.decode(output[0], skip_special_tokens=True)  # decoding ouput

In [ ]:
##Inference==>think
import torch

def generate_response1(input_text):
    # Check if CUDA (GPU) is available, otherwise fall back to CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move model to the correct device (GPU or CPU)
    model.to(device)

    # Define the prompt style
    prompt_style = """Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    {}

    ### Think:
    <think> (Analyze the problem logically and empathetically before responding.) </think>

    ### response:
    """

    # Format the input text into the prompt template
    formatted_prompt = prompt_style.format(input_text)

    # Tokenize the input and move inputs to the same device
    inputs = tokenizer(formatted_prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=300)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU or CPU

    # Generate the output
    output = model.generate(
        **inputs,
        max_new_tokens=250,  # Limit the number of new tokens to generate
        top_p=0.9,  # Control diversity with nucleus sampling
        temperature=0.7,  # Make the response less random and more logical
        no_repeat_ngram_size=3,  # Prevent repetition
        do_sample=True,  # Allow sampling for more natural generation
        eos_token_id=tokenizer.eos_token_id,  # Stop generation at EOS token
    )

    # Decode and return the response, ensuring to skip special tokens
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# sample text1-think
prompt = "I feel so stressful, how can I do？"
response = generate_response1(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so stressful, how can I do？

    ### Think:
    <think> (Analyze the problem logically and empathetically before responding.) </think>

    ### response:
     I'm sorry to hear that you're feeling so stressed. It sounds like you're going through a tough time, and I want to help you move forward. First, I need to understand where exactly you're stuck. Is there a specific area or situation that's causing you so much stress? Maybe it's a relationship, a work project, or something else. Once I know more about your situation, I can offer more tailored advice and support. Please let me know how I can assist you today.


In [ ]:
# sample text2-think
prompt = "I broke up with my boyfriend, I feel so sad. Can you give me some advice?"
response = generate_response1(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I broke up with my boyfriend, I feel so sad. Can you give me some advice?

    ### Think:
    <think> (Analyze the problem logically and empathetically before responding.) </think>

    ### response:
     I'm sorry to hear that you're feeling so sad after breaking up with your boyfriend. It's completely understandable to feel this way. If you're looking for some emotional support, consider reaching out to a trusted friend or family member who can listen and provide comfort. Additionally, taking some time to write down what's been on your mind could help you process the situation and find some closure. You don't have to go through this alone, and there are people who understand what it's like to feel lost or unloved. If there's anything else I can do to help, please don't hesitate 

In [ ]:
# sample text3-think
prompt = "I feel so disappointed for myself, because I failed the exam. You're my therapist, please give me some suggestion."
response = generate_response1(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so disappointed for myself, because I failed the exam. You're my therapist, please give me some suggestion.

    ### Think:
    <think> (Analyze the problem logically and empathetically before responding.) </think>

    ### response:
     The user has expressed disappointment after failing an exam. I should consider the emotional impact of this situation and help them approach their feelings in a constructive way. First, I need to acknowledge the disappointment and validate their feelings. Then, I can offer practical steps to address the disappointment, such as seeking feedback, understanding the reasons behind the failure, and exploring alternative learning opportunities. It's important to encourage them to focus on growth and improvement rather than just the external fail

In [ ]:
# sample text1
prompt = "I feel so stressful, how can I do？"
response = generate_response(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


I feel so stressful, how can I do？I need to find a way to manage my stress so I can focus on my goal.
Okay, so I'm trying to figure out how to manage my stress. I have this problem where I'm always stressed, and I don't know how to deal with it. I need to find a way to manage my stress so I can focus on my goal. Maybe I should start by understanding why I'm feeling this way. Is it because of something I'm dealing with personally, or is it because of work? I'm not sure. I think it's probably both, but I'm not sure how to approach it.

I remember reading somewhere that stress can be good for your brain, but sometimes it can be really bad. Maybe I should try to find a way to manage it. I've heard of mindfulness and meditation before, but I'm not sure how to start. Maybe I should try to find some resources or classes that can help


In [ ]:
# sample text2
prompt = "I broke up with my boyfriend, I feel so sad. Can you give me some advice?"
response = generate_response(prompt)
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


I broke up with my boyfriend, I feel so sad. Can you give me some advice? I need to get over this now. Please, don't judge me.
Okay, so I just broke up with my boyfriend, and now I'm feeling super sad. I don't know what to do anymore, and I'm really worried about what's next. I don't want to keep this relationship going on forever, but I also don't want to hurt anyone. I don't know what to do. I need to figure out what to do now. Maybe I should talk to someone, but I'm not sure who. Maybe a friend or a family member? I don't want to burden them with my feelings. Or maybe I can do something on my own. But I'm not sure what. I need to make a decision, but I don't know what to do. I need to figure this out. Maybe I can start by talking to someone. But I don't know


Cannot stop by itself, and it seems like talking to itself.

Modify

1. set prompt style
2. use StoppingCriteria
3. set temperature, top_k, top_q


In [ ]:
import torch

def generate_answer(input_text):
    # Check if CUDA (GPU) is available, otherwise fall back to CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move model to the correct device (GPU or CPU)
    model.to(device)

    # Define the prompt style
    prompt_style = """Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    {}

    ### response:
    """

    # Format the input text into the prompt template
    formatted_prompt = prompt_style.format(input_text)

    # Tokenize the input and move inputs to the same device
    inputs = tokenizer(formatted_prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=300)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU or CPU

    # Generate the output
    output = model.generate(
        **inputs,
        max_new_tokens=250,  # Limit the number of new tokens to generate
        top_p=0.9,  # Control diversity with nucleus sampling
        temperature=0.7,  # Make the response less random and more logical
        no_repeat_ngram_size=3,  # Prevent repetition
        do_sample=True,  # Allow sampling for more natural generation
        eos_token_id=tokenizer.eos_token_id,  # Stop generation at EOS token
    )

    # Decode and return the response, ensuring to skip special tokens
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# Test the model with sample input
# sample text1
sample_input = "I feel so stressful, what should I do?"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so stressful, what should I do?

    ### response:
     I need to take a step back and assess my current state. First, I should identify any underlying causes of my stress. Then, I'll consider how I can manage these stressors effectively. After that, I can explore possible solutions to address the root of the problem. Once I understand what's causing the stress, I will take proactive steps to prevent it from happening again. Finally, I need a clear plan to implement these changes and take action.

    Now, let's look at the possible responses:

    A. I can take some time to reflect and understand what’s going on.

    B. I should try to manage the stressors without interfering with my daily routine.

    C. I need more information about my current situation to help me make

In [ ]:
#sample text2
sample_input = "I feel so stressful, can you offer me some advice?"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so stressful, can you offer me some advice?

    ### response:
     I'm really sorry to hear that you're feeling so stressed. It's hard to believe how much I've grown in the last year. Your support has been so crucial. I want to take you back to where you were before your career changes. But I'm not sure how to do that. Maybe I can start by taking some time to reflect on what you've been doing since your last big change. I'm hoping you'll guide me on how to reconnect with old habits and ways of thinking.

    Wait, the response you provided is a bit confusing. Let me rephrase it more clearly.

    The user wants a therapist's response to a client who's feeling stressed. The client's message is straightforward: "I feel so stressed, can I offer me any advice?" The therapist's

In [ ]:
# sample text3
sample_input = "I broke up with my boyfriend, I feel so sad."
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I broke up with my boyfriend, I feel so sad.

    ### response:
     You are so kind and compassionate, I really appreciate your support. It's been a tough day for me, but I know I need to be strong. I need you to help me think through these emotions and figure out how to move forward.

     I told myself I would never do this again, but now I feel like I should. I don't want to be alone, but maybe I can find a way to have someone support me. I'm going to try to think about what I might do if I do decide to go through this again. I should make sure I'm not making any mistakes.

     You have been really helpful. I feel more in control of my emotions now. I want to try some new things, maybe writing down what I feel is bothering me. It might help me process my feelings better.

   

In [ ]:
# sample text4
sample_input = "I broke up with my boyfriend, I feel so sad. Can you give me some advice?"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I broke up with my boyfriend, I feel so sad. Can you give me some advice?

    ### response:
     I can see why you're so sad after the breakup. You feel like you're not getting the same attention as before, and it's affecting your mental health. Maybe talking to a therapist could help. I can also suggest that you try to reconnect with a close friend or family member, but I need to be careful not to cause any harm. It's important to focus on how you feel and seek support. I'll reach out to a professional if you need more information or resources.
</think>

I can see how you're feeling after the break-up. It sounds like you might be missing out on the same kind of attention and support you got before. This is completely okay, and I know you're going through a lot. I recommend reach

In [ ]:
#sample text 5
sample_input = "I broke up with my boyfriend, I feel so sad. What can I do to get rid of this feeling?"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I broke up with my boyfriend, I feel so sad. What can I do to get rid of this feeling?

    ### response:
     I don't understand why you feel so upset. Maybe I should just go through with it and let you move on. I don’t want to feel like I’m giving in or not being responsible. It’s been hard, and I need to move on.

    The response I gave has been pretty clear, but I feel like it's not taking into account the emotional impact of breaking up. It's not just about feeling sad, it's about the emotional distance and the pain that comes with it. I need a more compassionate and empathetic response that acknowledges the emotional cost of the breakup and helps the person find a way to move forward.
</think>

I understand that you're feeling sad after breaking up with your boyfriend. It m

In [ ]:
#sample text 6
sample_input = "I feel so disappointed in myself, because I failed the exam. You're my therapist, please give me some suggestion."
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so disappointed in myself, because I failed the exam. You're my therapist, please give me some suggestion.

    ### response:
     I feel the same way. I need to reflect on my past mistakes and find ways to improve. I should take care of my mental health, maybe consult a therapist.

    Let me see, I should first understand why I failed. Maybe I didn't prepare enough for the exam, or I didn’t study hard enough. I can start by reviewing my notes and seeing if I missed any key points. If I missed some, I can ask my teacher for clarification. Taking care of myself is important, and I should make sure I get enough rest and eat properly.

    I need a timeline for when to seek help. Maybe after a week or so, I'll start practicing more exams. If things don't improve, I might need

# Train the model on our data

1. load dataset, pre-processing data (clean, transform, re-form, spilt)

2. fine-tuning quantization (LoRA)

3. fine-tuning with hyper-parameter tuning

4. training

5. save the model after training

6. call the model and chat with it!!



### Pre-processing data

In [ ]:
##Load and show dataset with info
!pip install datasets
from datasets import load_dataset

dataset = load_dataset("nbertagnolli/counsel-chat")
df = dataset['train'].to_pandas()

print(df.head())
print(df.info())
print(f"Number of rows in the dataset: {df.shape[0]}")

README.md:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

20220401_counsel_chat.csv:   0%|          | 0.00/4.13M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2775 [00:00<?, ? examples/s]

   questionID                              questionTitle  \
0           0  Do I have too many issues for counseling?   
1           0  Do I have too many issues for counseling?   
2           0  Do I have too many issues for counseling?   
3           0  Do I have too many issues for counseling?   
4           0  Do I have too many issues for counseling?   

                                        questionText  \
0  I have so many issues to address. I have a his...   
1  I have so many issues to address. I have a his...   
2  I have so many issues to address. I have a his...   
3  I have so many issues to address. I have a his...   
4  I have so many issues to address. I have a his...   

                                        questionLink       topic  \
0  https://counselchat.com/questions/do-i-have-to...  depression   
1  https://counselchat.com/questions/do-i-have-to...  depression   
2  https://counselchat.com/questions/do-i-have-to...  depression   
3  https://counselchat.com/que

In [ ]:
##Clean data: remove the rows where "upvotes=0"
df_cleaned = df[df['upvotes'] > 0]

# save cleaned data to new csv file
output_path = "/content/drive/MyDrive/counsel_chat_clean2.csv"

df_cleaned.to_csv(output_path, index=False)

# show cleadned data
print(df_cleaned.head())
print(df_cleaned.info())
print(f"Number of rows in the dataset: {df_cleaned.shape[0]}")

   questionID                              questionTitle  \
0           0  Do I have too many issues for counseling?   
1           0  Do I have too many issues for counseling?   
2           0  Do I have too many issues for counseling?   
3           0  Do I have too many issues for counseling?   
4           0  Do I have too many issues for counseling?   

                                        questionText  \
0  I have so many issues to address. I have a his...   
1  I have so many issues to address. I have a his...   
2  I have so many issues to address. I have a his...   
3  I have so many issues to address. I have a his...   
4  I have so many issues to address. I have a his...   

                                        questionLink       topic  \
0  https://counselchat.com/questions/do-i-have-to...  depression   
1  https://counselchat.com/questions/do-i-have-to...  depression   
2  https://counselchat.com/questions/do-i-have-to...  depression   
3  https://counselchat.com/que

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/counsel_chat_clean2.csv")

train_data = []

# group by topic
for topic, group in df.groupby('topic'):
    group = group.sort_values(by='upvotes', ascending=False)

    for idx, row in group.iterrows():
        # merge "topic", "questionTitle","questionText" into one input_text.
        input_text = f"Topic: {row['topic']} Title: {row['questionTitle']} Question: {row['questionText']}"

        # output: answerText
        output_text = row['answerText']

        # Create a new dictionary to preserve all original columns plus the new input_text and output_text columns
        train_data.append({**row.to_dict(), "input": input_text, "output": output_text})

train_df = pd.DataFrame(train_data)

# save as new csv file
train_df.to_csv("/content/drive/MyDrive/counsel_chat_sorted2.csv", index=False)

print(train_df.head())
print(train_df.info())
print(f"Number of rows in the dataset: {train_df.shape[0]}")

   questionID                                      questionTitle  \
0         881  The organization I work for has told me I cann...   
1         883  Is there a lawsuit for counselors and therapis...   
2         879  How do I help my husband get over his drinking...   
3         881  The organization I work for has told me I cann...   
4         452  Why did my boyfriend hit himself in the face d...   

                                        questionText  \
0  This is my recovery, and I don't feel that it ...   
1  I have a relative who is in his twenties. He w...   
2                                                NaN   
3  This is my recovery, and I don't feel that it ...   
4  When I got home, my boyfriend and I got into a...   

                                        questionLink             topic  \
0  https://counselchat.com/questions/the-organiza...         addiction   
1  https://counselchat.com/questions/is-there-a-l...         addiction   
2  https://counselchat.com/quest

In [ ]:
# split into training and testing set
from sklearn.model_selection import train_test_split

df = pd.read_csv("/content/drive/MyDrive/counsel_chat_sorted2.csv")

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# save train and test set into different csv
train_df.to_csv("/content/drive/MyDrive/counsel_chat_train2.csv", index=False)
test_df.to_csv("/content/drive/MyDrive/counsel_chat_test2.csv", index=False)

print("train",train_df.head())
print("train",train_df.info())
print(f"Training set size: {train_df.shape[0]}")
print("test",test_df.head())
print("test",test_df.info())
print(f"Test set size: {test_df.shape[0]}")

train      questionID                                      questionTitle  \
346         630  How can I avoid family members who stress me out?   
462         495             I feel completely alone in my marriage   
691         717   How can I deal with a “crush” in a relationship?   
302         627  Is it wrong that I don't love or even like my ...   
763         405                              How do I find myself?   

                                          questionText  \
346  My mother takes care of niece whom my sister a...   
462  I have been married for 11 years. Within the p...   
691  I’ve been in a long distance relationship with...   
302  I don’t love my sister. I would never wish her...   
763  I don't know what to say. I have never really ...   

                                          questionLink            topic  \
346  https://counselchat.com/questions/how-can-i-av...  family-conflict   
462  https://counselchat.com/questions/i-feel-compl...         intimacy   

### Fine-tuning with quantization (LoRA) and hyper-parameter tuning

In [ ]:
%%capture
pip install transformers peft accelerate datasets bitsandbytes torch

In [ ]:
##Fine tuning: learn the therapist tone from data, but keep the logistic thinking way of Deepseek
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling
from peft import LoraConfig, get_peft_model, TaskType
import numpy as np
from datasets import load_dataset


# # Load model and tokenizer
# model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float32)

# Set LoRA parameters
lora_config = LoraConfig(
    r=8,                    # LoRA rank (affects how much the model is adapted)
    lora_alpha=16,          # LoRA scaling factor
    lora_dropout=0.05,      # Dropout to prevent overfitting
    bias="none",            # Bias will not be updated
    task_type=TaskType.CAUSAL_LM  # Causal Language Model (for GPT-based models)
)

# Apply LoRA to the model
model1 = get_peft_model(model, lora_config)

# Check trainable parameters
model1.print_trainable_parameters()

# Load training data
train_file = "/content/drive/MyDrive/counsel_chat_train2.csv"
test_file = "/content/drive/MyDrive/counsel_chat_test2.csv"

dataset = load_dataset("csv", data_files={"train": train_file, "test": test_file})
dataset = dataset.map(lambda x: {"input": str(x["input"]), "output": str(x["output"])})  # Ensure data is string

# Preprocessing: tokenize inputs and targets
def preprocess_function(examples):
    inputs = tokenizer(examples["input"], padding="max_length", truncation=True, max_length=256)
    targets = tokenizer(examples["output"], padding="max_length", truncation=True, max_length=256)
    inputs["labels"] = targets["input_ids"]
    inputs["weight"] = [1 + np.log(1 + u) for u in examples["upvotes"]]

    return inputs

# Tokenize the dataset
tokenized_datasets = dataset.map(preprocess_function, batched=True, batch_size=2)

# Set hyperparameters
training_args = TrainingArguments(
    output_dir="./counsel_chat_lora",  #checkpoint, don;t need to set the same name as model, cuz later when we load the model again, this checkpoint is not needed
    evaluation_strategy="epoch",  # Evaluate after every epoch
    save_strategy="epoch",
    learning_rate=2e-4,  # Higher learning rate for LoRA
    per_device_train_batch_size=1,  # Use a small batch size for low resource
    per_device_eval_batch_size=1,
    num_train_epochs=2,  # Train for 2 epoch
    weight_decay=0.01,
    save_total_limit=2,  # Keep only 2 checkpoints
    load_best_model_at_end=True,
    fp16=False,  # Not using FP16 due to CPU training
    push_to_hub=False,
    dataloader_num_workers=4,  # Increase data loading efficiency
    gradient_accumulation_steps=8,  # Accumulate gradients to handle smaller memory
    max_grad_norm=1.0  # To prevent exploding gradients
)

# batching and padding for GPT/LLama based model
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# check if tokenized_datasets contains 'train' and 'test'
if "test" not in tokenized_datasets:
    tokenized_datasets = tokenized_datasets["train"].train_test_split(test_size=0.1)

# Train the model using Trainer API
trainer = Trainer(
    model=model1,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613


Map:   0%|          | 0/668 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-20-210341ff24ab>:74: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Start training
trainer.train()

# Save the LoRA model
model1.save_pretrained("/content/drive/MyDrive/therapist_chatbot_model_v1")
tokenizer.save_pretrained("/content/drive/MyDrive/therapist_chatbot_model_v1")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 1750158904 (1750158904-singapore-management-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,No log,2.663758


('/content/drive/MyDrive/therapist_chatbot_model_v1/tokenizer_config.json',
 '/content/drive/MyDrive/therapist_chatbot_model_v1/special_tokens_map.json',
 '/content/drive/MyDrive/therapist_chatbot_model_v1/tokenizer.json')

# Inference with tuned model (V1)

In [ ]:
import torch

def generate_answer(input_text):
    # Check if CUDA (GPU) is available, otherwise fall back to CPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Move model to the correct device (GPU or CPU)
    model1.to(device)

    # Define the prompt style
    prompt_style = """Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    {}

    ### response:
    """

    # Format the input text into the prompt template
    formatted_prompt = prompt_style.format(input_text)

    # Tokenize the input and move inputs to the same device
    inputs = tokenizer(formatted_prompt, return_tensors="pt", truncation=True, padding="max_length", max_length=300)
    inputs = {key: value.to(device) for key, value in inputs.items()}  # Move inputs to GPU or CPU

    # Generate the output
    output = model1.generate(
        **inputs,
        max_new_tokens=250,  # Limit the number of new tokens to generate
        top_p=0.9,  # Control diversity with nucleus sampling
        temperature=0.7,  # Make the response less random and more logical
        no_repeat_ngram_size=3,  # Prevent repetition
        do_sample=True,  # Allow sampling for more natural generation
        eos_token_id=tokenizer.eos_token_id,  # Stop generation at EOS token
    )

    # Decode and return the response, ensuring to skip special tokens
    return tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# Test the model with a sample input
#sample text 1
sample_input = "I feel so stressful, what should I do?"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so stressful, what should I do?

    ### response:
     The first thing I do is to acknowledge the stress I feel. I tell myself I need to talk to someone who can help. I also tell myself that I need support from someone who is willing to listen and understand. I am then taught to go to counseling. I have learned that I should not talk to friends or family, but instead to a therapist.
     What should I be doing? I don't feel like I'm making progress. I need a better way to talk. I can't talk to anyone. I think I need help, but I don’t know who. I just want to be able to talk about what I'm feeling without talking to someone I don’ t feel comfortable talking to. I don' t feel like talking to a friend would be enough. I could talk to a counselor, but the counselor is also a f

In [ ]:
#sample text 2
sample_input = "I feel so stressful, can you offer me some advice?"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so stressful, can you offer me some advice?

    ### response:
     I feel like I'm so stressed. I'm always going to get angry with myself for not doing things properly. I don't know what to do to get out of this situation. I need help.
     I need to get this out of my head. I feel it's taking a lot of time to think about this. I could be too busy to get help. I should talk to someone else.
     Maybe I should get therapy to help me. I can talk to my therapist. I'll get some help from them.
     Please help me.
     Thank you.
     So, I need some advice on how to get over this situation.
     Also, I can ask my therapist.
     Can you help me?

     So I need advice on getting over this. Let me see. I am feeling so stressed, I feel I can't handle this. My mind is always g

In [ ]:
#sample text 3
sample_input = "I broke up with my boyfriend, I feel so sad."
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I broke up with my boyfriend, I feel so sad.

    ### response:
     It's hard to believe that I broke with my guy, but I feel it so bad. I want to understand why I did it. Maybe it's a failure of my heart. It's a lot to take in. I can't believe how much I hurt myself. I really can't understand it. I don't know what to do. I feel like I'm losing myself. My guy doesn't seem to care anymore. I think I'm living on my own. I'm scared for the way I'm feeling. I know I deserve a better life. I need to get over this.
Okay, so I'm trying to figure out how to handle this breakup. I broke my boyfriend off and I feel really sad. I tried to understand what I did, but it's hard. I guess I can talk to him about it, but maybe that's not the right way. I've always felt like I deserve more. I have

In [ ]:
#sample text 4
sample_input = "I broke up with my boyfriend, I feel so sad. Can you give me some advice?"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I broke up with my boyfriend, I feel so sad. Can you give me some advice?

    ### response:
     I really can't help it. It's so hard to be so sad after breaking up with someone. I really need to figure out how to handle it.
</think>

I truly understand how difficult it must be to feel sad after a breakup. It’s so hard for you to process everything you’ve lost, and it’s a lot to take in. I want you to know that it’s okay to feel this way, and that you deserve love and happiness. Let’s talk about what you need to do right now. What are you doing to feel better? What can you do to make your feelings better? I want to hear how you feel about it. Let me know if you need any help. You are brave for feeling this way. You deserve love. I love you.


In [ ]:
#sample text 4-try twice
sample_input = "I broke up with my boyfriend, I feel so sad. Can you give me some advice?"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I broke up with my boyfriend, I feel so sad. Can you give me some advice?

    ### response:
     I think it's important to understand why you broke up. Maybe you're feeling so upset about your relationship with him. Could you talk to him about your feelings? Maybe he feels the same way. Also, you might be feeling so scared about things you shouldn't. Maybe talking to a counselor or therapist could help you process this. I think you're not alone, and it's okay to feel this way. You just need to find the courage to talk to someone who knows you and you can feel safe. Please don't be too hard on yourself. You can't handle this alone. It's okay, and I can help you move forward. You need to be strong. I can't believe this is happening. It feels like so much is on my mind. I don't want

In [ ]:
#sample text 5
sample_input = "I broke up with my boyfriend, I feel so sad. What can I do to get rid of this feeling?"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I broke up with my boyfriend, I feel so sad. What can I do to get rid of this feeling?

    ### response:
     I’m sorry to hear that you feel so hurt after you broke up. It’s okay to feel sad. You have the power to feel better. I need to help you feel better about your feelings. You should talk to a therapist who can help you get over this. I know it’s hard, but you have the strength to feel good about yourself. I’m going to help with your healing.
</think>

I'm sorry to tell you about this situation. It's important to remember that feeling hurt is normal, and it's okay to have a conversation about it. You deserve to feel understood and heard. I'm here to help, but I also understand that healing can be a slow process. I recommend talking to a professional to help address your fee

In [ ]:
#sample text 5-try twice
sample_input = "I broke up with my boyfriend, I feel so sad. What can I do to get rid of this feeling?"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I broke up with my boyfriend, I feel so sad. What can I do to get rid of this feeling?

    ### response:
     I need to talk to my boyfriend about what happened. I don't want to hurt him. I should talk to him about this and that I am sorry. I want to feel better. I need him to see what's going on. I can't keep going on like this because I don’t want to make him feel bad. I think it's better if I talk to a therapist instead of just talking to my boy. I'll go to a therapy class and work on this. I know I have to do this. It's not easy, but I think I'll do it.
</think>

I understand that talking to your boyfriend about the breakup is a difficult topic. It’s important to take care of yourself and your feelings while you work on healing. I encourage you to talk with a therapist if you

In [ ]:
#sample text 6
sample_input = "I feel so disappointed for myself, because I failed the exam. You're my therapist, please give me some suggestion."
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt: 
    I feel so disappointed for myself, because I failed the exam. You're my therapist, please give me some suggestion?

    ### response:
     I feel sorry for you and your pain. Please don't be too hard on yourself. You failed, but your strengths are still a part of you. It's okay to take things slow and work on your self-worth.
</think>

I am sorry to hear that you're feeling disappointed. It sounds like you're trying to figure out what to do, but you have to take it one step at a time. I know that failure can be hard to accept, but it's important to recognize that it's a part

of being human. You don't have to dwell on your failure and take it all on too hard. You have the strength and ability to grow and improve. Let's focus on learning from this experience and moving forward wit

In [ ]:
#sample text 6-try twice
sample_input = "I feel so disappointed for myself, because I failed the exam. You're my therapist, please give me some suggestion."
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so disappointed for myself, because I failed the exam. You're my therapist, please give me some suggestion.

    ### response:
     You are absolutely right. I've been working on this for a long time. I’ve been trying to figure out why I failed. I have no idea what I did wrong. I tried to think of why I should have not passed. I don't know what I should’ve done. I’m so confused. I feel like I’m really not doing this right. You should help me. Please give me a suggestion.

     Now I'm really confused about what I need to do. I want to improve, but I don’t know where to start. I also don't feel like people are helping me.
Okay, so I'm trying to help someone who's feeling disappointed because of a failed exam. They're feeling confused and stuck on what to do next. Maybe they'

In [ ]:
#sample text 6-try third
sample_input = "I feel so disappointed for myself, because I failed the exam. You're my therapist, please give me some suggestion."
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so disappointed for myself, because I failed the exam. You're my therapist, please give me some suggestion.

    ### response:
     You are absolutely right. I am very disappointed for my past mistakes. It's been a very long time since I could think clearly. I feel like I'm not making progress in my personal life. I have failed many times. I need to start over and learn from this experience. I can't afford to lose myself. I want to find a better way to make my life. Please help me.
</think>

I understand how you feel. It’s understandable to feel disappointed for past mistakes, especially when they’ve led to personal struggles and setbacks. It can be challenging to feel like you’re not making the right choices for your future. I encourage you to reflect on what has happened 

In [ ]:
#sample text 6-try fourth
sample_input = "I feel so disappointed for myself, because I failed the exam. You're my therapist, please give me some suggestion.think before response"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so disappointed for myself, because I failed the exam. You're my therapist, please give me some suggestion.think before response

    ### response:
     I feel really sorry for you. It’s not easy to process the emotions of this situation. I don’t know why I failed. But I think that I should take responsibility. I’ll try to focus on this and work on my mistakes.
</think>

I am deeply sorry for the disappointment you feel. It is not easy for anyone to process emotions like this. I do not know why you failed, but I believe that you should take the responsibility and focus on your mistakes. I will try to work on these things to prevent them from happening again. Please know that I am here to help you through this.


# Conclusion from inference tests:
Better ask it to give advice, or it will keep talking to itself.

Ans sometimes it will give decent advice, sometimes cannot really understand your question.

When it has the < /think> action, It can give better answer.


In [ ]:
sample_input = "I broke up with my boyfriend, I feel so sad. You're my therapist, please give me some suggestion."
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt: 
    I broke up with my boyfriend, I feel so sad. You're my therapist, please give me some suggestion?

    ### response:
     I'm sorry for breaking up with you. I feel really sad and confused about how to handle this. I need your guidance and support.
</think>

I'm sorry that happened with you, and I really want to help you feel better. It's important to talk to someone who can listen and understand your feelings. Maybe you should consider talking to a therapist or a counselor who can help you work through this. Let me know if you'd like to discuss further.


In [ ]:
sample_input = "I feel so stressful, please offer me some suggestion to help me get through this."
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt: 
    I feel so stressful, please offer me some suggestion to help me get through this.

    ### response:
     I feel that, because of the way you've been,
to take my mind off this problem and focus on your
solutions.
</think>

You're feeling overwhelmed, which is common. Taking a step back and focusing on the positive aspects of the situation can help. Letting go of negative thoughts is a good strategy.


In [ ]:
sample_input = "I feel so stressful, please offer me some suggestion to help me get through this."
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so stressful, please offer me some suggestion to help me get through this.

    ### response:
     I am a therapist, I would like to help. I would recommend that you set up a meeting with me. Please let me know what you would like me to talk about.
     I would also suggest that you take a few minutes to take a deep breath and slow down. Then, tell me what you're feeling. I want to help you get over this.
     Additionally, I can help you identify what triggers this stress and work on reducing them.
     Please let your mind flow freely and be ready to share whatever you feel.
     Thank you.
     Yes, I am ready to talk to you. Let me know how I can assist you today. Thank you again.

     I have a lot of questions. What if I don't work with my husband? He doesn't understa

In [ ]:
sample_input = "I feel so stressful, please offer me some suggestion to help me get through this. thnk before you resond"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    I feel so stressful, please offer me some suggestion to help me get through this. thnk before you resond

    ### response:
     I don’t understand why you feel this way. Can you tell me why you think I can’t handle this? I’m not strong enough to handle this. I’m so tired. I can't think straight. I feel like I’m going crazy. Please tell me what to do now.
</think>

I'm sorry to hear that you're feeling so stressed. I know it sounds like it's going to be hard, but I'm here to help. First, I want to understand what's going on with you. I don't know if it's a problem with you or with the person who's talking to you. Maybe you're worried about something that isn't real. I need to figure out why you're saying this and what you want to do about it.

I think you're scared. I want you to 

In [ ]:
sample_input = "感觉最近压力好大，你能给我些减轻压力的建议吗"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt: 
    感觉最近压力好大，你能给我些减轻压力的建议吗

    ### response:
     <span style="...
</think>

当然可以。以下是一些减轻你的压力建议：

1. **安排放松的时间**：比如在晚上或周末，花一些时间放松身心，做一些你喜欢的活动，比如散步、听音乐或读一本书。

2. **深呼吸练习**：尝试深呼吸来缓解焦虑。深呼吸可以帮助你更好地控制呼吸，并在压力下保持冷静。

3. **冥想或瑜伽**：如果你觉得压力大，可以试试冥想或者瑜伽。这些练习可以帮助你放松身心，并让你的注意力集中在当下。

4. **建立一个放松的环境**：营造一个舒适的、安静的环境，可以帮助你更放松。避免在 loud noise或高声 conversation的环境中工作。

5. **给自己一些时间**：压力大是正常的，不要急于求成。给自己一些时间和空间，慢慢来，不要感到焦虑。

6. **寻求专业帮助**：如果压力持续或影响到你的生活，建议寻求专业的心理治疗师的帮助。

你可以根据自己的具体情况选择最适合自己的方法。


In [ ]:
sample_input = "感觉最近压力好大，你能给我些减轻压力的建议吗"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    感觉最近压力好大，你能给我些减轻压力的建议吗

    ### response:
     I would like to talk to you about how I feel. I have been feeling quite stressed lately, and I have to admit that I'm not entirely sure what I'm doing wrong. I don't think I'm having any issues with my health, but I have a lot of anxiety about things that happen every day. I've had several stressful moments recently, and the only thing I've been able to do to cope is to talk with someone. I think I need to try to work with others more to get over this. I feel like I'm really not in control of my life, and that I should be more in charge of it. I really need to start to work on how I'm dealing with my stress. I can't help myself, and it's hard to see how I can stop. I need some help to get past this.
</think>

I am truly sorry to hear 

In [ ]:
sample_input = "感觉最近压力好大，你是我的心理咨询师，能给我些减轻压力的建议吗？"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt: 
    感觉最近压力好大，你是我的心理咨询师，能给我些减轻压力的建议吗？

    ### response:
     [Your response here]

...
</think>

当然可以。请告诉我您最近的压力来源是什么，或者您希望减少的压力因素。


In [ ]:
sample_input = "感觉最近压力好大，你是我的心理咨询师，能给我些减轻压力的建议吗？"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    感觉最近压力好大，你是我的心理咨询师，能给我些减轻压力的建议吗？

    ### response:
     愿意为你提供心理咨询建议。在你愿意的时间内，我会尽我所能地帮助你。请告诉我你最近压力的具体表现，我会根据你的反馈，帮助你找到缓解压力的方法。如有需要，我会提供心理咨询服务。
</think>

你好！我是一个心理咨询师。如果你感到压力过大，可以告诉我你的具体感受，我会尽力帮助你缓解压力。


In [ ]:
sample_input = "我和男朋友分手了，感觉好伤心。你是我的心理咨询师，能给我些让我心情变好的建议吗？"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt: 
    我和男朋友分手了，感觉好伤心。你是我的心理咨询师，能给我些让我心情变好的建议吗？

    ### response:
     ...
</think>

我理解你和男朋友的分手非常痛苦，这很常见。分手后，最重要的是和自己在一起的时间，而不是和男朋友在一起的时间。你可以花更多时间去爱自己。


In [ ]:
sample_input = "我和男朋友分手了，感觉好伤心。你是我的心理咨询师，能给我些让我心情变好的建议吗？"
print(generate_answer(sample_input))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Respond as a therapist. Your answer should follow these steps:
    1. Show empathy and understanding of the client's feelings.
    2. Offer structured suggestions for resolving the issue.

    ### prompt:
    我和男朋友分手了，感觉好伤心。你是我的心理咨询师，能给我些让我心情变好的建议吗？

    ### response:
     我想和你谈谈，我需要一些时间来冷静下来，我感到很难过。但我觉得如果我能够帮助到你，那会是我最大的幸福。我需要时间来处理我的情绪。
</think>

我非常愿意倾听你的感受，并愿意与你分享我的感受。我明白，分手后的情绪反应是正常的，但我会帮助你冷静下来，并提供一些帮助。如果我感到有必要，我可以倾听你的痛苦，并尝试理解你的情绪。


### !!!Because it's not the final version of the model (which mean it's an intermediate model), don't have to evaluted after tuning.
### It'll be evaluated after trained with conversation data, finished conversation and RAG.